In [16]:
# 데이터셋 로더 설정

from torchvision import transforms, datasets
from torch.utils.data import DataLoader

transform = transforms.Compose([
        transforms.Resize((200,200)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
train_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/image_classification_dataset/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


In [20]:
import torch
import torch.nn as nn

# resnet50 정의

class ResidualBlock(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * self.expansion)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = torch.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self.make_layer(block, 64, layers[0], stride=1)
        self.layer2 = self.make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self.make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self.make_layer(block, 512, layers[3], stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def make_layer(self, block, out_channels, blocks, stride):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

def resnet50(num_classes=1000):
    return ResNet(ResidualBlock, [3, 4, 6, 3], num_classes)

resnet50_model = resnet50()




# 모델 정의
from torchvision.models import resnet50

class CustomResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(CustomResNet, self).__init__()
        self.resnet = resnet50(pretrained=False)
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.resnet(x)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#animalCF = CustomResNet(num_classes=10).to(device)
animalCF = resnet50().to(device)

In [21]:
# 모델 훈련

import torch.optim as optim

animalCF.train()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(animalCF.parameters(), lr = 0.001, momentum=0.9)
epochs = 100

for ep in range(epochs):
  running_loss = 0.0
  batch_count = 0
  for imgs, lbls in train_loader:
    batch_count += 1
    imgs = imgs.to(device)
    lbls = lbls.to(device)

    optimizer.zero_grad()
    out = animalCF(imgs)
    loss = criterion(out, lbls)
    loss.backward()
    optimizer.step()

    #print(f"{batch_count} batch loss: {loss.item()}")
    running_loss += loss.item()

  print(f"Epoch {ep+1}, Loss: {running_loss/len(train_loader):.4f}")

torch.save(animalCF.state_dict(), './model_weights.pth')


Epoch 1, Loss: 3.1482
Epoch 2, Loss: 1.8228
Epoch 3, Loss: 1.8215
Epoch 4, Loss: 1.8101
Epoch 5, Loss: 1.7969
Epoch 6, Loss: 1.8156
Epoch 7, Loss: 1.7907
Epoch 8, Loss: 1.7989
Epoch 9, Loss: 1.7996
Epoch 10, Loss: 1.8034
Epoch 11, Loss: 1.7574
Epoch 12, Loss: 1.7355
Epoch 13, Loss: 1.7816
Epoch 14, Loss: 1.7457
Epoch 15, Loss: 1.7381
Epoch 16, Loss: 1.6908
Epoch 17, Loss: 1.6921
Epoch 18, Loss: 1.7150
Epoch 19, Loss: 1.6920
Epoch 20, Loss: 1.6903
Epoch 21, Loss: 1.6198
Epoch 22, Loss: 1.6167
Epoch 23, Loss: 1.5811
Epoch 24, Loss: 1.5973
Epoch 25, Loss: 1.5541
Epoch 26, Loss: 1.5499
Epoch 27, Loss: 1.5319
Epoch 28, Loss: 1.5658
Epoch 29, Loss: 1.5363
Epoch 30, Loss: 1.5436
Epoch 31, Loss: 1.5390
Epoch 32, Loss: 1.4732
Epoch 33, Loss: 1.4245
Epoch 34, Loss: 1.4357
Epoch 35, Loss: 1.3794
Epoch 36, Loss: 1.3655
Epoch 37, Loss: 1.3396
Epoch 38, Loss: 1.2850
Epoch 39, Loss: 1.2940
Epoch 40, Loss: 1.2842
Epoch 41, Loss: 1.2402
Epoch 42, Loss: 1.1895
Epoch 43, Loss: 1.1651
Epoch 44, Loss: 1.12

In [23]:
# 모델 테스트

test_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/image_classification_dataset/test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

animalCF = resnet50().to(device)
animalCF.load_state_dict(torch.load('./model_weights.pth'))
animalCF.eval()

total = 0
correct = 0
with torch.no_grad():
    for imgs, lbls in test_loader:
        imgs = imgs.to(device)
        lbls = lbls.to(device)
        out = animalCF(imgs)
        _, predicted = torch.max(out, 1)
        total += lbls.size(0)
        correct += (predicted == lbls).sum().item()

print(f"Accuracy : {100*correct/total}%")

Accuracy : 42.833333333333336%
